Víctor González Kullmann u161806<br>
Albert Baito Pané u161812

## PART 1: Text Processing


In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
import json
import pycountry
import re
import string

In [2]:
#create a function to load the desired file containing the tweets
def load_data(file_name):
    with open(file_name) as f:
        tweets=json.load(f)
    return tweets
        
tweets = load_data("dataset_tweets_WHO.txt")

In [3]:
#function that transforms the iso lenguage in the complete name
def iso_leng_translate(leng):
    return pycountry.languages.get(alpha_2=leng).name.lower()

In [4]:
#get all the languages in the tweets of the datasets and get the stopwords of that language if available
def create_stopword_dict(tweets):
    #get languages
    lang=[]
    for i in range(len(tweets)):
        lang.append((tweets[str(i)]['lang']))
    languages = set(lang)
    print(languages)
    
    #search for the stopwords and save them in a dictionary for later usage
    lang_dict = {}
    for i in languages:     
        try:
            #transform ISO lenguage in complete name for convenience
            lang_dict [i] = set(stopwords.words(iso_leng_translate(i)))
            
        #if we don't have a stopwords available for that lenguage we just skip
        except:
            continue
        
        
    return lang_dict

stopwords_bylang = create_stopword_dict(tweets)

{'ja', 'ar', 'ru', 'fr', 'de', 'en', 'in', 'tl', 'uk', 'und', 'ps', 'es'}


In [5]:
def clean_data(tweets, stopwordsByLan):
    
    clean_texts = {}
    
    for i in range(len(tweets)):
        #get the tweet text
        og_tweet_text = (tweets[str(i)]['full_text']) 
        
        #lowercase to uniform format and split to get words
        og_tweet_text = og_tweet_text.lower()   
        og_tweet_text = og_tweet_text.split()
        
        #create our pattern to avoid removing #
        
        #remove punctuation
        tweet_text=[]
        for word in og_tweet_text:
            #maintain the links in the correct format
            if "https" not in word:
                #delete all punctuation conserving the # and @(in tweets are meaninguful)
                word = re.sub(r'[^\w\s#@]','', word)
                word = re.sub(r'_','',word)

            if word:
                tweet_text.append(word) 
        
        
        #get lenguage to filter stpowords
        tweet_lang = tweets[str(i)]['lang'].lower()
        #check availability of stopwors dict
        if tweet_lang in stopwordsByLan.keys():
            #if possible filter stopwords
            stop_words = set(stopwordsByLan[tweet_lang])
            clean_text = []
            for word in tweet_text:
                if word not in stop_words:
                    clean_text.append(word)
        
        #if stopwords are not available in some lenguage just let them 
        else:
            clean_text = tweet_text    

        
        #stem the words with the correct format for each lenguage
        try:
            stemmer = SnowballStemmer(iso_leng_translate(tweet_lang))
            clean_text = [stemmer.stem(word) for word in clean_text]
        
        except:
            pass
        
        clean_texts[tweets[str(i)]['id']] = clean_text
        
        
    return clean_texts

In [6]:
cleanDataset = clean_data(tweets, stopwords_bylang)
print(cleanDataset[1448215930178310144])

['intern', 'day', 'disast', 'risk', 'reduct', '#openwho', 'launch', 'multiti', 'core', 'curriculum', 'help', 'equip', 'compet', 'need', 'work', 'within', 'public', 'health', 'emerg', 'respons', 'start', 'learn', 'today', 'amp', '#ready4respons', 'https://t.co/hbffof0xkl', 'https://t.co/fgzy22rwus']


In [7]:
#function created to search for tweet id of a concrete lenguage
def tweet_lenguageSearcher(tweets, leng):
    for i in range(len(tweets)):
        if (tweets[str(i)]['lang']) == leng:
            print(tweets[str(i)]['id'])

In [8]:
tweet_lenguageSearcher(tweets, "es")

1448163383493136385
1447422682711068673
1447421758437564417
1447421622235942912
1446492798493003779
1437873802952708097
1437873628792639490
1436055322376949761
1436055263312818176
1426242406853353474
1425836042599374848
1425212438354571265
1424986407857238034
1423905220250243074
1423662605516804097
1422204917876461572
1421895908669763586
1411320867456434177
1409244087962775557


In [9]:
#function to plot the og tweet to check for the data processing correctness
def tweet_Searcher(tweets, id):
    for i in range(len(tweets)):
            if (tweets[str(i)]['id']) == id:
                print(tweets[str(i)]['full_text'])

In [10]:
tweet_Searcher(tweets, 1448215930178310144)

It's International Day for Disaster Risk Reduction

#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.

Start learning today &amp; be #Ready4Response:
👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS
